In [1]:
import torch, os, re, pandas as pd, json
# from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig
from datasets import Dataset, load_dataset
import sys

In [2]:
dataset_names = ['news', 'reddit', 'dialogue']
dataset_paths = {
    'news': '/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/initial_datasets/news/sharded',
    'reddit': '/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/initial_datasets/reddit/sharded',
    'dialogue': '/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/initial_datasets/dialogue'
}
max_target_lenghts = {
    'news':175,
    'reddit':100,
    'dialogue':100
}

bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

special_tokens_dict = {
    'eos_token': '<|EOS|>', 
    'bos_token': '<|endoftext|>', 
    'pad_token': '<|pad|>'
}

text_column = 'document'
summary_column = 'summary'

In [3]:
# def pretty_print(text, max_len_line=100):
#     words = text.split(' ')
#     len_line = 0
#     line = ''
#     for w in words:
#         if w == '\n':
#             print(line)
#             line = ''
#             continue
#         if (len(line) + len(w)) > max_len_line:
#             print(line)
#             line = ''
#         line += ' ' + w
#     print(line)

In [4]:
if torch.backends.mps.is_available():  
    dev = "mps" 
else:  
    dev = "cpu"  
device = torch.device(dev)  

In [5]:
# # We load the model
# base_model = GPT2LMHeadModel.from_pretrained('gpt2')
# # options: ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
# base_model.num_parameters

In [6]:
# base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# print('Words in vocabulary: ', base_tokenizer.vocab_size)
# vocabulary = base_tokenizer.get_vocab()
# vocabulary['Hi']
# text = "Hi, I'm Victor and I work as a Data Scientist"
# base_tokenizer.tokenize(text)

In [7]:
# text_ids = base_tokenizer.encode(text, return_tensors = 'pt')
# text_ids

In [8]:
# text = "I work as a data scientist"
# text_ids = base_tokenizer.encode(text, return_tensors = 'pt')

# generated_text_samples = base_model.generate(
#     text_ids,
# )
# base_tokenizer.decode(generated_text_samples[0], skip_special_tokens=True)

In [9]:
# device = torch.device("mps")
# device

In [124]:
# the eos and bos tokens are defined


special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}

# We load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# the new token is added to the tokenizer
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.padding_side = 'left'

# the model config to which we add the special tokens
config = AutoConfig.from_pretrained('gpt2', 
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    output_hidden_states=False)

# the pre-trained model is loaded with the custom configuration
model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)

# the model embedding is resized
model.resize_token_embeddings(len(tokenizer))


Embedding(50259, 768)

In [128]:
dataset_name = 'reddit'

dataset = load_dataset('Data/initial_datasets/reddit', data_files='validation_data.csv')['train'].select([i for i in range(25)])
print(dataset)

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(f"No. added tokens ={num_added_toks}")
target_length = max_target_lenghts[dataset_name]
print(f"target_length = {target_length}")
separator = " TL;DR "
separator_ids = tokenizer.encode(separator)
print(f"separator_ids = {separator_ids}")
separator_length = len(separator_ids)
print(f"separator_length = {separator_length}")
separator_attention_mask = [1] * separator_length
print(f"separator_attention_mask = {separator_attention_mask}")
global_max_length = tokenizer.model_max_length
print(f"global_max_length = {global_max_length}")
src_length = global_max_length - separator_length - target_length
print(f"src_length = {src_length}")

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 25
})
No. added tokens =0
target_length = 100
separator_ids = [24811, 26, 7707, 220]
separator_length = 4
separator_attention_mask = [1, 1, 1, 1]
global_max_length = 1024
src_length = 920


In [102]:
def preprocess_function(examples):
    input_ids = []
    attention_mask = []
    for i in range(len(examples)):
        model_input = torch.full((1, tokenizer.model_max_length), tokenizer.pad_token_id)
        doc_ids = tokenizer.encode(special_tokens_dict['bos_token'] + ' ' + examples[text_column][i], padding='do_not_pad', max_length=src_length, truncation=True, return_tensors='pt')
        summ_ids = tokenizer.encode(" TL;DR " + examples[summary_column][i], padding='do_not_pad', max_length=target_length-1, truncation=True, return_tensors='pt')
        in_put = torch.cat((doc_ids, summ_ids, torch.Tensor([[tokenizer.eos_token_id]])), dim=1)
        model_input[0, model_input.shape[1]-in_put.shape[1]:] = in_put
        
        input_ids.append(model_input)
        attention_mask.append(torch.where(model_input!=tokenizer.pad_token_id, 1, 0))
        
    return {
        'input_ids':input_ids,
        'attention_mask':attention_mask
    }

In [103]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=3,
    remove_columns=dataset.column_names,
)
# tokenized_dataset.set_format("pt", columns=["input_ids", "attention_mask"], output_all_columns=True) 
tokenized_dataset.set_format("pt", columns=["input_ids"], output_all_columns=True) 

Map (num_proc=3):   0%|          | 0/10 [00:00<?, ? examples/s]

In [105]:
for i in tokenized_dataset['input_ids']:
    print("*********"*5, "\n", tokenizer.decode(i[0]))
    print()

********************************************* 
 <|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>

In [17]:
# path = './test'

# training_args = TrainingArguments(
#     output_dir=path,          # output directory
#     num_train_epochs=1,              # total # of training epochs
#     per_device_train_batch_size=1,  # batch size per device during training
#     per_device_eval_batch_size=1,   # batch size for evaluation
# )

# data_collator = DataCollatorForLanguageModeling(
#         tokenizer=tokenizer,
#         mlm=False
#     )
# trainer = Trainer(
#     model=model,                         # the instantiated  Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     data_collator=data_collator,
#     train_dataset=tokenized_dataset
# )
# trainer.train()

# trainer.save_model()
# tokenizer.save_pretrained(path)

# trainer.evaluate()


In [259]:
# def preprocess_generation_function(examples):
#     print(len(examples))
#     inputs = []
#     targets = []
#     for i in range(len(examples)):
#         print(examples['summary'][i])
#         inputs.append(bos + examples[text_column][i])
#         targets.append(examples[summary_column][i])
    
#     model_inputs = tokenizer(inputs, padding='max_length', max_length=src_length, truncation=True)
#     model_inputs['input_ids'] = [input_ids + separator_ids for input_ids in model_inputs['input_ids']]
#     model_inputs['attention_mask'] = [attention_mask + separator_attention_mask for attention_mask in model_inputs['attention_mask']]
#     target_ids = tokenizer(targets, padding='max_length', max_length=target_length, truncation=True)
    
#     model_inputs['label_ids'] = target_ids['input_ids']
#     return model_inputs

In [294]:
def preprocess_generation_function(example):
    inputs = bos + ' ' + example[text_column]
    targets = example[summary_column]
    # print(inputs)
    # print(targets)
    
    model_inputs = tokenizer(inputs, padding='max_length', max_length=src_length, truncation=True)
    model_inputs['input_ids'] = model_inputs['input_ids'] + separator_ids
    model_inputs['attention_mask'] = model_inputs['attention_mask'] + separator_attention_mask
    target_ids = tokenizer(targets, padding='max_length', max_length=target_length, truncation=True)
    
    # print("model_inputs - ", model_inputs)
    # print("target_ids - ", target_ids)
    
    model_inputs['label_ids'] = target_ids['input_ids']
    return model_inputs

In [295]:
len(dataset['summary'][5:30])

20

In [300]:
tokenized_dataset = dataset.map(
    preprocess_generation_function,
    batched=False,
    num_proc=1,
    # remove_columns=dataset.column_names,
    load_from_cache_file=0
)
tokenized_dataset.set_format("pt", columns=["input_ids", "attention_mask", "label_ids"], output_all_columns=True) 

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [301]:
tokenized_dataset

Dataset({
    features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'label_ids'],
    num_rows: 25
})

In [302]:
output = model.generate(
    input_ids=tokenized_dataset['input_ids'],
    attention_mask=tokenized_dataset['attention_mask'],
    do_sample=False,
    max_new_tokens = target_length
)

In [304]:
for i in range(len(output)):
    print("\n", "***********"*20)
    print(tokenizer.decode(tokenized_dataset['input_ids'][i]))
    print("\n", "_______________________" *20)
    print(tokenizer.decode(output[i]))


 ****************************************************************************************************************************************************************************************************************************
<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>

In [100]:
output_text = tokenizer.decode(output[0])
print(output_text)
# list_output = list(output[0])
# len(list_output[list_output.index(tokenizer.pad_token_id):])

<|endoftext|>obligatory "this was a few years ago"... i was 14, and my parents had left me home alone for the weekend. of course, being the the lazy teenager i was, i couldn't bring myself to make real food. top ramen was on the menu for lunch. 

everything was going smoothly until i tipped the stupid bowl over on the counter with a spoon. all of the wonderful beef flavored broth was now on the counter, but i had a realization. "hey, i just cleaned the kitchen a few hours ago... i might be able to salvage this." i grabbed a towel and wiped up most of the broth, then wrung it out back into the bowl. genius! 

nope. the fuck up... i went to the couch, took my first delicious spoonful... and immediately vomited everywhere. as i quickly realized, the towel i had used to rescue my ramen was also the towel i had used to clean the kitchen... with bleach. yup, way to go. i had just ruined my appetite, my ramen, my couch, and my weekend in one fell swoop. i didn't live it down for a few months 

In [101]:
input_ids = tokenizer.encode("Hi my name is april. TL;DR ", return_tensors='pt')
print(input_ids)
output = model.generate(
    input_ids,
    max_length=global_max_length,
    # max_new_tokens = target_length
    early_stopping=True
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

tensor([[17250,   616,  1438,   318, 46593,   346,    13, 24811,    26,  7707,
           220]])


/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/langgen/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


KeyboardInterrupt: 

In [94]:
print(tokenizer.decode(output[0]))

Hi my name is april. TL;DR <|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>